## Features variabelen uitbreiden 

In [1]:
# libraries importeren 
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# dataset inlezen 
df = df = pd.read_csv('sap_storing_data_hu_project.csv', low_memory=False)

In [3]:
#Doel : en bepaald probleem op een bepaalde locatie op een bepaald tijdstip, een voorspelling te maken van de verwachte reparatietijd


#Belangrijke kolommen bepalen

# Geeft een goede definitie van de probleem 
#oorz_code : feature - nominaal 
#oorz_groep :feauture - nominaal 
#stm_prioriteit :feature - ordinaal 

# Deze kolom geven een duidelijke gebied waar de aannemer werkt.
#stm_contractgeb_mld of  stm_contractgeb_gst: C Gebied waar de aannemer werkt. - nominaal 
    # Definieer de speciefieke gebied waar de aannemer zit.
# geocode vallen in contract gebied , maar niet alle

# De tijdstip van een storing weergeven. 
# tijd van de melding 
data = df[['stm_oorz_groep', 'stm_oorz_code', 'stm_prioriteit','stm_contractgeb_mld','stm_contractgeb_gst','stm_sap_meldtijd', 'stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_ddt']].copy()

In [ ]:
data.loc[:, 'reparatietijd'] = (pd.to_datetime(data['stm_fh_ddt']) -  pd.to_datetime(data.loc[:,'stm_aanntpl_dd']  +" "+ data['stm_aanntpl_tijd'])).dt.total_seconds().div(60)

In [5]:
data.head()

,stm_oorz_groep,stm_oorz_code,stm_prioriteit,stm_contractgeb_mld,stm_contractgeb_gst,stm_sap_meldtijd,stm_aanntpl_dd,stm_aanntpl_tijd,stm_fh_ddt,reparatietijd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ONR-RIB,133.0,9.0,NaN,81.0,09:00:00,NaN,00:00:00,02/01/2006 09:00:00,NaN
2,ONR-DERD,143.0,9.0,NaN,37.0,12:35:00,NaN,00:00:00,02/01/2006 13:26:00,NaN
3,ONR-DERD,142.0,9.0,NaN,32.0,16:40:00,NaN,00:00:00,02/01/2006 17:20:00,NaN
4,ONR-DERD,142.0,9.0,NaN,35.0,22:30:00,NaN,00:00:00,02/01/2006 22:36:00,NaN


In [6]:
data = data.drop(['stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_ddt'], axis= 1)

We hebben te maken met een continu target variabel en categorical feature variabelen 

In [8]:
#Features variabelen  2:
features = data[['stm_oorz_groep', 'stm_oorz_code', 'stm_prioriteit', 'stm_contractgeb_gst', 'stm_sap_meldtijd']]
features.head()

,stm_oorz_groep,stm_oorz_code,stm_prioriteit,stm_contractgeb_gst,stm_sap_meldtijd
0,NaN,NaN,NaN,NaN,NaN
1,ONR-RIB,133.0,9.0,81.0,09:00:00
2,ONR-DERD,143.0,9.0,37.0,12:35:00
3,ONR-DERD,142.0,9.0,32.0,16:40:00
4,ONR-DERD,142.0,9.0,35.0,22:30:00


Voor de tijd van de melding moeten de tijden worden afgerond naar uren.

In [16]:
df2  = df[[ 'stm_contractgeb_gst', 'stm_sap_meldtijd' ]].copy()
df2.head()

,stm_contractgeb_gst,stm_sap_meldtijd
0,NaN,NaN
1,81.0,09:00:00
2,37.0,12:35:00
3,32.0,16:40:00
4,35.0,22:30:00


In [17]:
# De dimensie van de df2 bekijken
df2.shape

(898526, 2)

In [18]:
#datatypes bepalen
df2.dtypes

stm_contractgeb_gst    float64
stm_sap_meldtijd        object
dtype: object

In [20]:
#kijken hoeveel Nan waardes er zijn.
df2.isnull().sum()

stm_contractgeb_gst    223
stm_sap_meldtijd        38
dtype: int64

In [70]:
# bereken aantal rijen die wordt verloren bij Nan waardes, kan denk ik gewoon verwijderd worden, is maar een heel klein aantal.
print('Rijen verloren door Nan contractgebied : {}%'.format((df2['stm_contractgeb_gst'].isnull().sum()/898526)*100))
print('Rijen verloren door Nan meldtijd  : {}%'.format((df2['stm_sap_meldtijd'].isnull().sum()/898526)*100))
print('Totaal aantal rijen verloren : {} %'.format(df2.isnull().sum().sum()/898526*100))

Rijen verloren door Nan contractgebied : 0.02481842484246421%
Rijen verloren door Nan meldtijd  : 0.004229148627863857%
Totaal aantal rijen verloren : 0.02904757347032807 %


In [ ]:
#de rijen daadwerkelijk verwijderen.

In [30]:
# Nog even kijken of er codes in het excel bestand staan die hier missen en andersom.
df2['stm_contractgeb_gst'][df2['stm_contractgeb_gst'].notnull()].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 50.,
       51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63.,
       64., 70., 71., 81., 82., 83., 99.])

In [36]:
len(df2['stm_contractgeb_gst'].unique())

60

In [71]:
# 50., 53., 54., 55., 56., 57., 62., 64., 82., 83.
a = len(df2[df2['stm_contractgeb_gst'] == 50])
a

348

In [72]:
b = len(df2[df2['stm_contractgeb_gst'] == 53])
b

15073

In [73]:
c = len(df2[df2['stm_contractgeb_gst'] == 54])
c

5926

In [74]:
d = len(df2[df2['stm_contractgeb_gst'] == 55])
d

5253

In [75]:
e = len(df2[df2['stm_contractgeb_gst'] == 56])
e

2898

In [76]:
f =len(df2[df2['stm_contractgeb_gst'] == 57])
f

289

In [77]:
g = len(df2[df2['stm_contractgeb_gst'] == 62])
g

9377

In [78]:
h = len(df2[df2['stm_contractgeb_gst'] == 64])
h

4051

In [79]:
i = len(df2[df2['stm_contractgeb_gst'] == 82])
i

111

In [68]:
j = len(df2[df2['stm_contractgeb_gst'] == 83])
j

694

In [69]:
totaal = a + b + c + d + e + f + g + h + i + j
totaal

44020

In [80]:
# aantal waardes dat verloren zou gaan als we het verwijderen. (in procenten)
44020/898526*100

4.899134805225447

In [89]:
df3 = df['stm_sap_meldtijd'].str.strip(":").astype(int).copy()
df3.head()

ValueError: invalid literal for int() with base 10: ''